In [ ]:
import json
import tweepy
import sys
import traceback
from datetime import datetime, timezone
from kafka import KafkaProducer

In [ ]:
consumer_key    = 'T3bxHQI78bQSMxhryPQk4rgMs'
consumer_secret = 'uZEP6eLjUGGkS4YtRvm3PISnpMlho7vniHKeoZMF2NVu9B6rpe'
access_token    = '1207006565074640897-c761jqaeRPkTPeyjBFxlPqsirDhYQ1'
access_secret   = 'qmJiaHbct4hshX4mcVIx2BuKCPdo8xndNPb7bOLes4MOs'

In [ ]:
# Probably you can compare the performance of opposing hashtags insuch way
#Keywords = "#OwenJonesIsAWankerDay #SolidarityWithOwenJonesDay"

In [ ]:
Keywords = 'Israel'

In [ ]:
class TwitterStreamListener(tweepy.StreamListener):
    def __init__(self):
        self.producer = KafkaProducer(bootstrap_servers='localhost:9092',
                                      key_serializer=str.encode, 
                                      value_serializer=lambda v: json.dumps(v).encode('utf-8'))

        self.datefmt = "%a %b %d %H:%M:%S %z %Y"
        self.topic = "TweeterArchive"
        self.tweets = []
        self.rawdata = [] 
        self.count = 0
        self.ltweet = ''        
    def on_data(self, data):
        # data is the full *tweet* json data
        api_events = json.loads(data)
      
        # save raw json data in memory array for debugging
        self.rawdata.append(data)
        
        #filter out retweets
        if not api_events['text'].startswith('RT'):
            
            # Gathering relevant values
            # Tweet-related values
            event_fields = [ 'id', 'text','created_at', 'geo', 'coordinates', 'place',
                          "quote_count", "reply_count", "retweet_count", "favorite_count" ]
            twitter_events = {k:v for k,v in api_events.items() 
                              if k in event_fields}    
            
            tweet_key, tweet_created_at = self.date_fmt_convert(twitter_events['created_at'])
                        
            twitter_events['created_at'] = tweet_created_at
            
            # When tweets are longer than 140 symbols, the 'text' field gets truncated, and the
            # full text is in ['extended_tweet']['full_text']
            if 'extended_tweet' in api_events:
                twitter_events['text'] = api_events['extended_tweet']['full_text']

            # User-related values
            user_fields = ['id', 'name', 'screen_name','created_at', 'location', 'url',
                         'protected', 'verified', 'followers_count', 'friends_count',
                         'listed_count', 'favourites_count', 'statuses_count', 'withheld_in_countries']
            user_events = {k:v for k,v in api_events['user'].items() 
                           if k in user_fields}
           
            user_events['created_ym'], user_events['created_at'] = self.date_fmt_convert(user_events['created_at'],fmt = "%Y%m") 

            twitter_events['user_id'] = user_events['id'] 
            twitter_events['user_followers'] = user_events['followers_count']
            
            events = {'users':user_events, 'tweets': twitter_events }                

            # save processed dictionary in memory array for debugging
            self.tweets.append((tweet_key,events))
            # print last tweet to console on the same line
            self.ltweet = events['tweets']['text'][:50].replace('\n','')
             
            self.send_to_kafka(str(tweet_key), events)                 
            
        print('\r tweets received: '+ str(self.count) + '| last relevant tweet: ' + self.ltweet,end='')                
        self.count+=1        

    def send_to_kafka(self, key, data):  
        
        # send data to kafka topic(s)
        #self.producer.send('TweeterArchive', events)
        #self.producer.send('TweeterData', events) 
        self.producer.send(self.topic, key = key, value = data)
        self.producer.flush()
        
    def date_fmt_convert(self, date, fmt = "%Y%m%d%H%M"):        
        # Convert twitter datetime format such as 
        # "Sat Jan 4 11:39:13 +0500 2019"
        # to partitioning-compatible format and shift it to utc, so: 
        # key (int): 201901040639, timestamp (int): 1547420603
        date_time_obj = datetime.strptime(date, self.datefmt)
        date_time_obj.astimezone(tz=timezone.utc)
        return date_time_obj.strftime(fmt), int(date_time_obj.timestamp())


    def on_error(self, status_code):
        if status_code == 420:
            return False

In [ ]:
def initialize():
    try:
        auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
        auth.set_access_token(access_token, access_secret)
        api = tweepy.API(auth)

        stream = TwitterStreamListener()
        twitter_stream = tweepy.Stream(auth=api.auth, listener=stream)
        twitter_stream.filter(track=[Keywords], languages=['en']) # -filter:retweets
    except (KeyboardInterrupt, Exception) as oops:
        print("\noops " + repr(oops))
        #print(''.join(traceback.format_tb(oops.__traceback__)))
        return stream, oops

In [ ]:
stream,oops = initialize()

In [ ]:
oops

## Sandbox 
below are cells that I played with to test portions of code


In [ ]:
int.encode(3423)

In [ ]:
dir(str.encode('434234'))

In [ ]:
wat = str.encode('434g34')
wat.isalnum()

In [ ]:
wat

In [ ]:
i = 99991231235
#i=1
i

In [ ]:
len(str(i))

In [ ]:
encode = lambda x : x.to_bytes(5, byteorder='big')
decode = lambda x : int.from_bytes(x, byteorder='big')

In [ ]:
ii = encode(i)

In [ ]:
ii

In [ ]:
decode(encode(i))

In [ ]:
decode(encode(i)) == i

In [ ]:
int.from_bytes(hm, byteorder='big')

In [ ]:
hm.isalnum()

In [ ]:
int(hm, base =2, byteorder='big')

In [ ]:
traceback.extract_tb(oops.__traceback__ )

In [ ]:
datefmt = "%a %b %d %H:%M:%S %z %Y"

date = "Sun Jan 13 23:03:23 +0000 2019"

def date_fmt_convert(date, fmt = "%Y%m%d%H%M"):        
    # Convert twitter datetime format such as 
    # "Sat Jan 4 11:39:13 +0500 2019"
    # to partitioning-compatible format and shift it to utc, so: 
    # int: 201901040639, int: 
    date_time_obj = datetime.strptime(date, datefmt)
    date_time_obj.astimezone(tz=timezone.utc)
    return int(date_time_obj.strftime(fmt)), int(date_time_obj.timestamp())

In [ ]:
date_fmt_convert(date)

In [ ]:
tweets = stream.tweets

In [ ]:
rawdata = stream.rawdata

In [ ]:
wat = tweets
print(len(wat))
wat4 = wat[1]
wat4

In [ ]:
wat4[1]

In [ ]:
wat4 = wat4.replace('1210547837035073536', '12105478370350735364305983475023948606723056927602')
wat4

In [ ]:
watj = json.loads(wat4)

In [ ]:
watj

In [ ]:
id = wat4[1]['user']['id']

In [ ]:
wat4

In [ ]:
print(json.loads(wat[i])['extended_tweet']['full_text'])

In [ ]:
print(json.dumps(json.loads(wat[i])))

In [ ]:
    date_time_obj = datetime.strptime(date, self.datefmt)
        date_time_obj = date_time_obj.astimezone(tz=timezone.utc)
        return date_time_obj.strftime("%Y-%m-%d_%H-%M")


In [ ]:
date = wat4['tweet']['created_at']

In [ ]:
 date_time_obj = datetime.strptime(date, "%Y-%m-%d_%H-%M")

In [ ]:
type(date_time_obj.timestamp())

In [ ]:
api_event = json.loads(wat4)

In [ ]:
api_event['retweeted']

In [ ]:
api_event

In [ ]:
str_event = json.dumps(api_event, ensure_ascii=False)
str_event

In [ ]:
with open('./test.json','w') as f:
    str_event = json.dump(api_event,f, ensure_ascii=False)
str_event

In [ ]:
print(bytes(str_event, 'utf-8' ))

In [ ]:
wat[4]['retweeted_status']#['extended_tweet']['full_text']